<a href="https://colab.research.google.com/github/ansonkwokth/TableTennisPrediction/blob/main/inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/ansonkwokth/TableTennisPrediction.git
%cd TableTennisPrediction

Cloning into 'TableTennisPrediction'...
remote: Enumerating objects: 169, done.
remote: Counting objects: 100% (169/169), done.
remote: Compressing objects: 100% (145/145), done.
remote: Total 169 (delta 80), reused 42 (delta 17), pack-reused 0 (from 0)
Receiving objects: 100% (169/169), 680.10 KiB | 2.61 MiB/s, done.
Resolving deltas: 100% (80/80), done.
/content/TableTennisPrediction


In [2]:
import pandas as pd
from tabulate import tabulate
from utils import data_loader as dl

import numpy as np
from model.Elo import Elo
from model.ModifiedElo import ModifiedElo
from model.ensemble import BaggingRatingSystem

import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm

import copy

import warnings
warnings.filterwarnings('ignore')

In [3]:
# GAME = 'TTStar'
# GAME = 'TTCup'
# GAME = 'SetkaCup'
GAME = 'SetkaCupWomen'
# GAME = 'LigaPro'


In [4]:
match GAME:
    case 'TTStar':
        years = [2020, 2021, 2022, 2023, 2024]
    case 'TTCup':
        years = [2020, 2021, 2022, 2023, 2024]
    case 'SetkaCup':
        years = [2020, 2021, 2022, 2023, 2024]
    case 'SetkaCupWomen':
        years = [2020, 2021, 2022, 2023, 2024]
    case 'LigaPro':
        years = [2022, 2023, 2024]
    case _:
        raise ValueError("Invalid game selected.")


text_data_game = dl.load_game_data(GAME, years, '../')
text_data = {
    year: text_data_game[year] for year in years
}
df = dl.create_game_dfs(GAME, years, text_data)

Loading ..//SetkaCupWomen2020.txt
Loading ..//SetkaCupWomen2021.txt
Loading ..//SetkaCupWomen2022.txt
Loading ..//SetkaCupWomen2023.txt
Loading ..//SetkaCupWomen2024.txt


In [5]:
# Generate ID indices for each pair of rows in the DataFrame
idx_lt = [i for i in range(len(df) // 2) for _ in range(2)]
df['ID'] = idx_lt  # Assign to the 'ID' column

# Reset the DataFrame index to ensure it's sequential
df.reset_index(drop=True, inplace=True)

# Get unique players and store them in player_lt
player_lt = df['Player'].unique()



In [6]:
def format_to_array(df: pd.DataFrame) -> np.ndarray:

    # info_col = ['ID', 'Round', 'Datetime', 'Game', 'Date', 'Time']
    info_col = ['Round', 'Datetime', 'Game', 'Date', 'Time']
    col = [item for item in df.columns if item not in info_col]

    df[[c for c in col if "Set" in c]] = df[[c for c in col if "Set" in c]].astype(float)
    X = df[col].values.reshape(-1, 2, len(col))
    return X

In [7]:
X_all = format_to_array(df)

In [8]:
modelMElo = ModifiedElo()
modelMElo.fit(X_all)


Training model: 100%|██████████| 34579/34579 [00:01<00:00, 17965.72it/s]


# Print

In [9]:
def find_player(name):
    p_lt = []
    for k, v in modelMElo.params.items():
        if name in k:
            print(k)
            p_lt.append(k)
    return p_lt

In [10]:
def find_OU(player1, player2):
    set1_dt = modelMElo.predict_set_config(player1, player2)[-1]
    set2_dt = modelMElo.predict_set_config(player2, player1)[-1]

    set_sum_dt = {}
    for k1, v1 in set1_dt.items():
        v2 = set2_dt[k1]
        set_sum_dt[k1] = v1 + v2



    p_O = 0
    p_U = 0
    for k, v in set_sum_dt.items():
        if k > line:
            p_O += v
        else:
            p_U += v
    return p_O, p_U

In [11]:
df_lt = []

In [80]:
p1_lt = find_player("Ase")
print("-"*30)
p2_lt = find_player("Andr")

Aseeva I.
------------------------------
Andreeva L.
Andryushchenko V.


In [81]:
player1 = p1_lt[0]
player2 = p2_lt[1]
player1, player2

('Aseeva I.', 'Andryushchenko V.')

In [82]:

date = "Jan 29"


line = 18.5
time = "14:30"


found_p1, found_p2, p_game = modelMElo.predict_game(player1, player2)
found_p1, found_p2, p_set = modelMElo.predict_set(player1, player2)
p_O, p_U = find_OU(player1, player2)
winner = player1 if p_game > 0.5 else player2
p_set_win = p_set if winner == player1 else 1 - p_set
p_game_win = p_game if winner == player1 else 1 - p_game
p_OU = p_O if p_O > p_U else p_U
OU = "O" if p_O > p_U else "U"

In [83]:
if (found_p1 and found_p2):
    df_row = pd.DataFrame({
                         "Time": [time],
                         "Player1": [player1], "Player2": [player2], "Winner": [winner],
                         "Set": [f"{p_set_win*100:.1f}%"], "Game": [f"{p_game_win*100:.1f}%"],
                         "O/U": [f"{OU} {line}: {p_OU*100:.1f}%"]})

    print(tabulate(df_row, headers = 'keys', tablefmt = 'psql', showindex=False))


+--------+-----------+-------------------+-------------------+-------+--------+---------------+
| Time   | Player1   | Player2           | Winner            | Set   | Game   | O/U           |
|--------+-----------+-------------------+-------------------+-------+--------+---------------|
| 14:30  | Aseeva I. | Andryushchenko V. | Andryushchenko V. | 57.7% | 64.2%  | O 18.5: 51.3% |
+--------+-----------+-------------------+-------------------+-------+--------+---------------+


In [84]:
p_U, p_O

(0.48701063394522137, 0.5129893660547786)

In [85]:
df_lt.append(df_row)

In [86]:
df_print = pd.concat(df_lt)
df_print.index = range(len(df_lt))
print(tabulate(df_print, headers = 'keys', tablefmt = 'psql', showindex=False))
# print(tabulate(df_print, headers = 'keys', tablefmt = 'psql'))

+--------+-------------------+-------------------+-------------------+-------+--------+---------------+
| Time   | Player1           | Player2           | Winner            | Set   | Game   | O/U           |
|--------+-------------------+-------------------+-------------------+-------+--------+---------------|
| 07:30  | Kurtenko A.       | Yureneva S.       | Kurtenko A.       | 59.6% | 67.6%  | O 18.5: 50.9% |
| 08:00  | Harmash N.        | Andryushchenko V. | Andryushchenko V. | 76.8% | 91.5%  | U 18.5: 56.5% |
| 09:00  | Kurtenko A.       | Andryushchenko V. | Andryushchenko V. | 58.4% | 65.4%  | O 18.5: 51.2% |
| 10:30  | Kurtenko A.       | Aseeva I.         | Aseeva I.         | 50.8% | 51.5%  | O 18.5: 51.9% |
| 11:30  | Harmash N.        | Yureneva S.       | Yureneva S.       | 63.8% | 74.6%  | U 18.5: 50.1% |
| 12:30  | Harmash N.        | Aseeva I.         | Aseeva I.         | 69.5% | 83.1%  | U 18.5: 52.3% |
| 13:00  | Andryushchenko V. | Yureneva S.       | Andryushchenk